In [ ]:
# set it up to autoreload updates
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from utils import get_date_from_df
import json

In [ ]:
with open('configs/pv_info.json', 'r') as f:
    pv_info = json.load(f)
    f.close()

with open('configs/model_info.json', 'r') as f:
    model_info = json.load(f)
    f.close()

In [ ]:
from configs.ref_config import ref_point

ref_point = ref_point[0]
ignored_features = ['ACCL:IN20:400:L0B_ADES', 'Pulse_length', 'FBCK:BCI0:1:CHRG_S']

def overwrite_ignored_features(time_series, model_info, pv_info, ref_point, ignored_features):
    for pvname in ignored_features:
        sim_name = pv_info['pv_name_to_sim_name'][pvname]
        # ref points are in sim values so we need to convert to pv_units
        feature_loc = model_info['loc_in'][sim_name]
        reference_val = pv_info['sim_to_pv_factor'][sim_name] * ref_point[feature_loc]
        time_series[pvname] = reference_val
        print(f'reset {pvname} to {reference_val}')
    return time_series

In [ ]:
cols = ['QUAD:IN20:525:BACT','OTRS:IN20:571:XRMS', 'OTRS:IN20:571:YRMS']
subset_val_data = pd.read_pickle('archive_data/val_df.pkl')

# find the date of the scan data and read in the full time range
date = get_date_from_df(subset_val_data)
val_data = pd.read_pickle(f'archive_data/injector_{date}.pkl')

# update the raw validation data with the changes we make to the training data itself
val_data['CAMR:IN20:186:R_DIST'] = np.sqrt(val_data['CAMR:IN20:186:XRMS'].values**2 + val_data['CAMR:IN20:186:YRMS'].values**2)
val_data = overwrite_ignored_features(val_data, model_info, pv_info, ref_point, ignored_features)

print(all(item in val_data.columns for item in subset_val_data.columns))

In [ ]:
val_data.info()

In [ ]:
start = val_data.index[0]
for _ in range(7):
    end = start + timedelta(hours=1)
    fig, ax = plt.subplots(2,1, figsize=(12,8))
    ax[0].plot(val_data[str(start):str(end)][cols[0]], label=cols[0])
    ax[1].plot(val_data[str(start):str(end)][cols[1]], label=cols[1])
    ax[1].plot(val_data[str(start):str(end)][cols[2]], label=cols[2])

    ax[1].legend()
    fig.tight_layout()
    plt.show()
    start = end

In [ ]:
scans = val_data[(val_data['OTRS:IN20:571:YRMS'] < 600) & (val_data['OTRS:IN20:571:YRMS'] > 10) & (val_data['OTRS:IN20:571:XRMS'] > 10)]['2021-11-18 03:00:00-08:00': '2021-11-18 04:35:00-08:00']
fig, ax = plt.subplots(2,1,figsize=(30,8))
ax[0].plot(scans[cols[0]], label=cols[0])
ax[1].plot(scans[cols[1]], label=cols[1])
ax[1].plot(scans[cols[2]], label=cols[2])

ax[1].legend()
fig.tight_layout()
plt.show()

In [ ]:
start = scans.index[0]
for _ in range(4):
    end = start + timedelta(hours=0.5)
    fig, ax = plt.subplots(2,1, figsize=(12,8))
    ax[0].plot(scans[str(start):str(end)][cols[0]], label=cols[0])
    ax[1].plot(scans[str(start):str(end)][cols[1]], label=cols[1])
    ax[1].plot(scans[str(start):str(end)][cols[2]], label=cols[2])

    ax[1].legend()
    fig.tight_layout()
    if len(scans[str(start):str(end)][cols[0]]) > 10:
        plt.show()
    start = end

In [ ]:
from plotting import interactive_plot, plot_scans

In [ ]:
interactive_plot(scans, cols=cols)

In [ ]:
scan_periods = [

    # ("2021-12-09 01:32:15-0800", "2021-12-09 01:33:02-0800"),
    # ("2021-12-09 01:41:36-0800", "2021-12-09 01:42:17-0800"),
    ("2021-11-18 03:02:00-0800", "2021-11-18 03:07:00-0800"),
    ("2021-11-18 03:37:00-0800", "2021-11-18 03:40:00-0800"),
    ("2021-11-18 04:15:36-0800", "2021-11-18 04:18:17-0800"),
]
plot_scans(scans, scan_periods, otr_name='OTRS:IN20:571', quad_name='QUAD:IN20:525:BACT')
for i, (start, end) in enumerate(scan_periods):
    scan_df = scans[str(start):str(end)]
    scan_df.to_pickle(f'archive_data/val_scan_{i}.pkl')

In [ ]:
features = ['CAMR:IN20:186:R_DIST',
 'Pulse_length',
 'FBCK:BCI0:1:CHRG_S',
 'SOLN:IN20:121:BACT',
 'QUAD:IN20:121:BACT',
 'QUAD:IN20:122:BACT',
 'ACCL:IN20:300:L0A_ADES',
 'ACCL:IN20:300:L0A_PDES',
 'ACCL:IN20:400:L0B_ADES',
 'ACCL:IN20:400:L0B_PDES',
 'QUAD:IN20:361:BACT',
 'QUAD:IN20:371:BACT',
 'QUAD:IN20:425:BACT',
 'QUAD:IN20:441:BACT',
 'QUAD:IN20:511:BACT',
 'QUAD:IN20:525:BACT']

In [ ]:
scan_df.head()

In [ ]:
test = scans[scan_periods[0][0]:scan_periods[0][1]].copy()
oidx = test.index
nidx = pd.date_range(oidx.min(), oidx.max(), freq='1s')
res = test.reindex(oidx.union(nidx)).interpolate(method='linear',order=2).reindex(nidx)
# interpolated = test_s.interpolate(method='linear')
# interpolated
res
fig, ax = plt.subplots()
ax.scatter(res['QUAD:IN20:525:BACT'], res['OTRS:IN20:571:XRMS'])
ax.scatter(res['QUAD:IN20:525:BACT'], res['OTRS:IN20:571:YRMS'])
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(res['QUAD:IN20:525:BACT'], res['OTRS:IN20:571:XRMS'])
ax.scatter(res['QUAD:IN20:525:BACT'], res['OTRS:IN20:571:YRMS'])
plt.show()

In [ ]:
# for start, end in scan_periods:
    # scan_data = scans[str(start):str(end)]
plot_scans(scans, scan_periods, otr_name='OTRS:IN20:571', quad_name='QUAD:IN20:525:BACT')

In [ ]:
fig, ax = plt.subplots(2,1,figsize=(30,8))
ax[0].plot(scans[cols[0]], label=cols[0])
ax[1].plot(scans[cols[1]], label=cols[1])
ax[1].plot(scans[cols[2]], label=cols[2])

ax[1].legend()
fig.tight_layout()
plt.show()
